In [2]:
import pandas as pd

In [5]:
pwd

'/Users/vasudev/Desktop/Projects/imc_prosperity/prosperity-imc25/rounds/round2/eda'

In [14]:
day_price_1_df = pd.read_csv("./round-2-island-data-bottle/prices_round_2_day_1.csv", delimiter=";")
day_price_0_df = pd.read_csv("./round-2-island-data-bottle/prices_round_2_day_0.csv", delimiter=";")
day_price_m1_df = pd.read_csv("./round-2-island-data-bottle/prices_round_2_day_-1.csv", delimiter=";")

# merge all the dataframes
merged_df = pd.concat([day_price_m1_df, day_price_0_df, day_price_1_df])

# df for each product
croissants_df = merged_df[merged_df['product'] == 'CROISSANT']
jams_df = merged_df[merged_df['product'] == 'JAMS']
djembe_df = merged_df[merged_df['product'] == 'DJEMBE']
pic1_df = merged_df[merged_df['product'] == 'PICNIC_BASKET_1']
pic2_df = merged_df[merged_df['product'] == 'PICNIC_BASKET_2']

In [13]:
merged_df.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-1,0,CROISSANTS,4304,112,NaN,NaN,NaN,NaN,4305,112,NaN,NaN,NaN,NaN,4304.5,0.0
1,-1,0,JAMS,6670,66,6669.0,137.0,NaN,NaN,6671,66,6672.0,137.0,NaN,NaN,6670.5,0.0
2,-1,0,SQUID_INK,2005,1,2002.0,31.0,NaN,NaN,2006,31,NaN,NaN,NaN,NaN,2005.5,0.0
3,-1,0,PICNIC_BASKET1,59284,20,59283.0,18.0,NaN,NaN,59294,2,59295.0,36.0,NaN,NaN,59289.0,0.0
4,-1,0,PICNIC_BASKET2,30606,20,30605.0,18.0,NaN,NaN,30612,20,30613.0,18.0,NaN,NaN,30609.0,0.0


# PICNIC BASKET 1